In [93]:
### web crawling
# - 정적 페이지 웹크롤링 : 페이지의 데이터가 변경될때 URL변경 O (HTML)
# - 동적 페이지 웹크롤링 : 페이지의 데이터가 변경될때 URL변경 X (JSON)
# requests모듈 : 브라우저의 URL입력시 서버에서 데이터를 다운받아 화면에 출력
import requests as rs
import pandas as pd

print(rs.__version__)
print(pd.__version__)

2.25.1
1.4.3


In [97]:
# https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2

# Naver Stock Data
# Kospi, Kosdaq, USD

# 동적페이지 실습

class naver_stock():
    def stock_price(page=1,page_size=10,code='KOSPI'):
        """
        page = {int} puts page
        page_size = {int} 
        code = {string} whitch site do you want? (KOSPI/KOSDAQ)
        """
        url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={page_size}&page={page}"
        return naver_stock.return_pd(url, ["localTradedAt", "closePrice"])
    def stock_exchange(page=1, page_size=10, code='FX_USDKRW'):
        """
        page = {int} puts page
        page_size = {int} 
        code = {string} whitch foreign currency exchange?
        """
        url = f"https://api.stock.naver.com/marketindex/exchange/{code}/prices?page={page}&pageSize={page_size}"
        return naver_stock.return_pd(url, ["localTradedAt", "closePrice"])
    def return_pd(url, list):
        data = rs.get(url).json()
        return pd.DataFrame(data)[list]

# print(naver_stock.stock_price(1,4))
# print(naver_stock.stock_price(1,4,"KOSDAQ"))
print(naver_stock.stock_exchange(1, 60, "FX_USDKRW").head(10))
print(naver_stock.stock_exchange(1, 60, "FX_JPYKRW").head(10))

  localTradedAt closePrice
0    2022-08-03   1,311.50
1    2022-08-02   1,312.00
2    2022-08-01   1,305.00
3    2022-07-29   1,304.50
4    2022-07-28   1,302.00
5    2022-07-27   1,314.00
6    2022-07-26   1,311.00
7    2022-07-25   1,312.00
8    2022-07-22   1,310.00
9    2022-07-21   1,315.00
  localTradedAt closePrice
0    2022-08-03     984.57
1    2022-08-02     997.83
2    2022-08-01     989.42
3    2022-07-29     979.32
4    2022-07-28     968.21
5    2022-07-27     957.83
6    2022-07-26     960.55
7    2022-07-25     960.86
8    2022-07-22     963.06
9    2022-07-21     950.87


In [95]:
df = naver_stock.stock_price(1,60).copy()
df['kosdaq'] = naver_stock.stock_price(1,60,'KOSDAQ')['closePrice']
df["usd"] = naver_stock.stock_exchange(1, 60,'FX_USDKRW')['closePrice']
df = df.rename(columns = {'closePrice' : 'kospi'})

df['kospi'] = df['kospi'].apply(lambda data : float(data.replace(",","")))
df['kosdaq'] = df['kosdaq'].apply(lambda data : float(data.replace(",","")))
df['usd'] = df['usd'].apply(lambda data : float(data.replace(",","")))
print(df.head(5))
print(df.dtypes)

print(df[['kospi', 'kosdaq', 'usd']].corr())

  localTradedAt    kospi  kosdaq     usd
0    2022-08-03  2457.92  814.53  1311.1
1    2022-08-02  2439.62  804.34  1312.0
2    2022-08-01  2452.25  807.61  1305.0
3    2022-07-29  2451.50  803.62  1304.5
4    2022-07-28  2435.27  798.32  1302.0
localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object
           kospi    kosdaq       usd
kospi   1.000000  0.984038 -0.877288
kosdaq  0.984038  1.000000 -0.820035
usd    -0.877288 -0.820035  1.000000


In [86]:
# call by 실습

a = [1, 2, 3]
b = a          # 얕은 복사 call by reference
c = a.copy()   # 깊은 복사 call by value
print(a,b,c)

a[1] = 4
print(a,b,c)   # 이러한 결과가 나오는 이유는 c언어마냥 주소값이 존재하기 떄문에 a, b가 가리키는것은 [1,2,3]의 주소값.따라서 b도 변화한것. 
               # 그냥 데이터 다룰때는 메모리 더 써도 안전한 copy()사용 권장.

[1, 2, 3] [1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


In [92]:
# lambda 비교

# 함수 3개 사용, 메모리 3칸 사용
def plus(n1,n2):
    return n1+n2
def minus(n1,n2):
    return n1-n2
def calc(func, n1, n2):
    return func(n1, n2)

print(calc(plus, 1, 2), calc(minus, 1, 2))

# plus_lambda = lambda n1, n2:n1+n2
# print(plus_lambda(1,2))

# 함수 1개 사용, 메모리 1칸 사용
def calc(func, n1, n2):
    return func(n1, n2)

calc(lambda n1, n2: n1 + n2, 1, 2), calc(lambda n1, n2: n1 - n2, 1, 2)

3 -1


(3, -1)